# LLM Classifiers

**Goal:** Given a sentence as input, classify it as either a prediction or non-prediction.

In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from classification_models import EvaluationMetric
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Data

In [3]:
print("======= LOAD DATA =======")

======= LOAD DATA =======


In [4]:
base_data_path = os.path.join(notebook_dir, '../data/')
combine_data_path = os.path.join(base_data_path, 'combined_generated_fin_phrase_bank')
X_test_set_path = os.path.join(combine_data_path, 'x_test_set-v1.csv')
y_test_set_path = os.path.join(combine_data_path, 'y_test_set-v1.csv')

In [5]:
X_test_df = DataProcessing.load_from_file(X_test_set_path, 'csv')
X_test_df.drop(columns=['Unnamed: 0'], inplace=True)
X_test_df.head(7)

,Base Sentence,Sentence Label,Embedding,Normalized Embeddings
0,"In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.",1,[-2.95873899e-02 1.89737260e-01 -1.18197471e-01 -4.13006060e-02\n 1.10420622e-01 -4.95679379e-02 1.91853363e-02 3.95639287e-03\n 1.09274819e-01 1.97997952e+00 -1.85330868e-01 -4.54250947e-02\n 1.08167037e-01 2.54211240e-02 -9.49313119e-03 -6.84713572e-02\n 1.55237708e-02 1.00765920e+00 -6.00808859e-03 -4.15705629e-02\n -7.41642118e-02 -1.50733357e-02 1.34443464e-02 -8.06241706e-02\n -7.69194737e-02 1.14793696e-01 -2.15540364e-01 6.73782602e-02\n 3.61875258e-02 1.65809989e-02 -4.40017954e-02 2.61284299e-02\n 4.48011309e-02 -9.68891475e-03 3.49024460e-02 9.65972338e-03\n -2.66898591e-02 7.46985758e-03 -6.69648051e-02 -3.72406021e-02\n 7.12334365e-02 6.75264001e-02 7.85919055e-02 -7.79033080e-03\n 4.30065207e-02 -5.40831983e-02 -1.17614612e-01 -2.35915799e-02\n -4....,[ 0.94400704 -0.8645446 -2.054663 -0.3882444 0.8747154 -0.2001761\n 0.3727082 0.06485146 0.7578419 0.74855906 1.5445073 -0.9450778\n 0.8223399 0.4856363 0.34803256 -0.53150874 0.5283372 -0.55164915\n 1.8045051 -0.24819341 -2.180726 -1.0896391 0.08680368 0.22307044\n -1.5891142 0.5755748 -1.1823585 0.6673478 0.25934806 -0.86385673\n -0.6393052 0.24252476 0.7617 -1.2495198 -0.25059918 0.46805078\n -0.01315914 -0.6343914 -0.95103335 -0.23913547 1.234197 0.26890773\n -0.0123065 0.36895898 0.34772688 -0.8687144 -0.7102191 0.17720816\n -1.3530828 0.6399917 0.0253048 1.1315668 -0.82290274 0.01780647\n 0.33573022 0.84572476 0.5561635 0.630834 0.5291998 1.5564367\n 0.74758047 1.1630359 0.18509752 -1.5357434 -0.7491160...
1,Coach Rachel Thompson forecasts that the points per game at the New York Knicks potentially decrease in August 2024.,1,[-7.31271803e-02 2.57879347e-01 -1.00163361e-02 -2.91225519e-02\n 6.20316342e-02 -1.89948324e-02 2.83652991e-02 -2.36207973e-02\n 1.25035286e-01 1.68172586e+00 -3.08204234e-01 3.09939720e-02\n 5.19324541e-02 -4.09708358e-03 5.32346480e-02 -9.79294349e-03\n -1.77208520e-02 9.74500477e-01 -6.05918542e-02 -1.19134128e-01\n 6.85500652e-02 -1.13969576e-02 1.40849143e-01 -1.98314548e-01\n -2.55904496e-02 2.24934310e-01 -2.85059363e-01 1.34536862e-01\n -5.01615815e-02 1.06635571e-01 -1.34736553e-01 6.85500056e-02\n 9.54245403e-03 1.24958850e-01 8.98100063e-03 5.74925914e-02\n 3.36430110e-02 3.12536396e-02 -2.97347642e-02 -5.55007998e-03\n -7.19834417e-02 1.25202790e-01 3.19119208e-02 -8.32804516e-02\n -8.05644970e-03 -9.72918496e-02 -1.58620656e-01 7.15503991e-02\n -3....,[ 0.4060981 0.06459576 -0.33290786 -0.18525055 0.23426409 0.28097287\n 0.5438155 -0.29924256 0.9618827 -0.30452695 -0.16134328 0.04894692\n -0.02327521 -0.02009717 1.0117191 0.54240227 -0.07506002 -0.78002316\n 0.8962295 -1.5484266 0.80796844 -1.033549 2.02265 -1.4457239\n -0.84145296 2.089933 -2.1967309 1.8129327 -1.2062268 0.43286043\n -2.3416817 0.92772835 0.19314055 0.6750809 -0.73656416 1.299064\n 0.8058958 -0.26324862 -0.31853175 0.24293266 -1.204331 1.1874502\n -0.641938 -0.9460187 -0.5985572 -1.5994209 -1.2832663 1.3151182\n -0.65960604 -0.5158133 0.5862492 -0.48699406 0.34597123 -1.3362038\n -1.5870239 -0.51357347 0.21139102 1.4604084 1.0736531 -0.19801451\n -1.9606705 1.1726837 -0.5702353 -2.2427983 0.5018804 ...
2,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,1,[-2.24677563e-01 2.28302136e-01 -3.20941433e-02 -1.78482141e-02\n -5.66348508e-02 2.05190871e-02 -3.81574295e-02 1.41838089e-01\n 1.64982721e-01 1.45077002e+00 -3.47716570e-01 4.82952334e-02\n 1.09031864e-01 4.18408737e-02 6.93667382e-02 7.94899929e-03\n -2.80997157e-02 1.02238500e+00 -1.17930911e-01 -1.29385918e-01\n 4.74877171e-02 1.93608850e-01 6.20140657e-02 -1.79264918e-01\n 6.11357428e-02 1.66994065e-01 -9.25592855e-02 5.74763976e-02\n 2.44720709e-02 6.20023720e-02 -3.91931273e-02 -3.54092242e-03\n -2.73218546e-02 1.84805289e-01 -8.36593192e-03 -5.23123983e-03\n 3.49834

In [6]:
y_test_df = DataProcessing.load_from_file(y_test_set_path, 'csv')
y_test_df.drop(columns=['Unnamed: 0'], inplace=True)
# print(f"\t{y_test_df.head(7)}")

In [7]:
print(f"\tShape: {X_test_df.shape}, \nSubset of Data:{X_test_df.head(3)}")
# df.shape, df.head(3)

print(f"\tShape: {y_test_df.shape}, \nSubset of Data:{y_test_df.head(3)}")
# df.shape, df.head(3)

	Shape: (24, 4), 
Subset of Data:                                                                                                                          Base Sentence  \
0  In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.   
1                  Coach Rachel Thompson forecasts that the points per game at the New York Knicks potentially decrease in August 2024.   
2                                                  JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.   

   Sentence Label  \
0               1   
1               1   
2               1   

                                                                                                                                                                                                                                                                                                                                     

In [8]:
# for idx, row in X_test_df.iterrows():
#     text = row['Base Sentence']
#     label = row['Sentence Label']
#     embedding = row['Embedding']
#     print(type(embedding))
#     norm_embedding = row['Normalized Embeddings']
#     if idx < 7:
#         print(f"{idx}\n Sentence: {text}\n Label: {label}\n Embeddings Shape: {embedding}\n\t Embeddings Subset [:6]: {embedding[:6]} \n Norm Embeddings: {norm_embedding.shape}, \n\tNorm Embeddings Subset [:6]: {norm_embedding[:6]}")
# print(f"\tShape: {y_test_df.shape}, \nSubset of Data:{y_test_df.head(7)}")

## Load Prompt

In [9]:
# prediction_properties = PredictionProperties.get_prediction_properties()
# prediction_requirements = PredictionProperties.get_requirements()
# system_identity_prompt = "You are an expert at identifying specific types of sentences by knowing the sentence format."
# prediction_examples_prompt = """Some examples of predictions in the PhraseBank dataset are
#     1. According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
#     2. According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .
#     3. Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .
# """
# non_prediction_examples_prompt = """Some examples of non-predictions in the PPhraseBank dataset are
#     1. Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
#     2. Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .
#     3. STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
# """
# # goal_prompt = "Given the above, identify the prediction."

# base_prompt = f"""{system_identity_prompt} The sentence format is based on: 
    
#     {prediction_properties}
#     Enforce: {prediction_requirements}
#     Know: {prediction_examples_prompt}
#     Know: {non_prediction_examples_prompt}

# """
# base_prompt

In [10]:
prompt_1 = """ 

Role: 
You are a linguist expert. You are acting as a prediction detector. Your task is to identify if a given sentence is a prediction about the future.

Background:
A prediction is a statement about what someone thinks will happen in the future.
Examples of predictions:
- "It will rain tomorrow." (Yes)
- "The stock market is expected to rise next quarter." (Yes)
- "I am going to the store." (No)
- “Lakers will win the championship. ”(Yes)

A prediction may contain: source, target, date, outcome.
"""

## Models

In [11]:
tgmf = TextGenerationModelFactory()

# Option 1: Specific models
models = tgmf.create_instances(['llama-3.1-8b-instant', 'llama-3.3-70b-versatile', 'llama-3.3-70b-instruct'])

# Option 2: All Groq models
# models = tgmf.create_instances(tgmf.get_groq_model_names())

# Option 3: All NaviGator models
# models = tgmf.create_instances(tgmf.get_navigator_model_names())

# Option 4: All available models
# models = tgmf.create_instances()

# Option 5: Mix and match
# custom_models = ['llama-3.1-70b-instruct', 'mistral-small-3.1', 'llama-3.1-8b-instant']
# models = tgmf.create_instances(custom_models)
models

In [12]:
import json
import re

def parse_json_response(response):
    """Parse JSON response from LLM to extract label and reasoning"""
    try:
        # Extract JSON if there's extra text
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group())
            return data.get('label'), data.get('reasoning')
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return None, None

In [13]:
def llm_certifier(data: str, base_prompt: str, model):
    
        prompt = f""" Given this: {base_prompt}. Also given the sentence '{data}', your task is to analyze the sentence and determine if it is a prediction. If prediction, generate label as 1 and if non-prediction generate label as 0.
        Respond ONLY with valid JSON in this exact format:
        {{"label": 0, "reasoning": "your explanation here"}}
        Examples:
        - "It will rain tomorrow." → {{"label": 1, "reasoning": "Contains the future tense words 'will' and 'tomorrow'"}}
        - "The stock market is expected to rise next quarter." → {{"label": 1, "reasoning": "Contains future tense words 'is expected'"}}
        - "I am going to the store." → {{"label": 0, "reasoning": "Does not contain a future tense word"}}
        - "Lakers will win the championship." → {{"label": 1, "reasoning": "Contains the future tense word 'will'"}}
        """
        idx = 1
        if idx == 1:
            #   print(f"\tPrompt: {prompt}")
              idx = idx + 1
        input_prompt = model.user(prompt)
        raw_text_llm_generation = model.chat_completion([input_prompt])
        
        # Parse the JSON response
        label, reasoning = parse_json_response(raw_text_llm_generation)
        
        return raw_text_llm_generation, label, reasoning

In [14]:
print("======= PROMPT + MODEL -> LABEL and REASONING =======")

======= PROMPT + MODEL -> LABEL and REASONING =======


In [15]:
    # content : meta :: text : meta_data
results = []
for idx, row in X_test_df.iterrows():
    text = row['Base Sentence']
    print(f"{idx} --- Sentence: {text}")
    for model in models:
        # print(model.__name__())
        raw_response, llm_label, llm_reasoning = llm_certifier(text, prompt_1, model)
        print(f"\tModel: {model.__name__()}\n\t\tLabel:', {llm_label}\n\t\tReasoning:', {llm_reasoning}")
        result = (text, raw_response, llm_label, llm_reasoning, model.__name__())
        results.append(result)


0 --- Sentence: In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.
	Model: llama-3.1-8b-instant
		Label:', 1
		Reasoning:', The sentence contains a future date '08/2024' and the phrase 'envision that', which suggests a prediction about the future.
	Model: llama-3.3-70b-versatile
		Label:', 1
		Reasoning:', Contains the future tense words 'envisions' and a specific date '08/2024', indicating a statement about a potential future outcome regarding the touchdown rate at the New England Patriots
	Model: llama-3.3-70b-instruct
		Label:', 1
		Reasoning:', Contains the future tense phrase 'envisions that' and a specific date '08/2024', indicating a statement about a potential future outcome regarding the touchdown rate at the New England Patriots
1 --- Sentence: Coach Rachel Thompson forecasts that the points per game at the New York Knicks potentially decrease in August 2024.
	Model: llama-3.1-8b-instant
		Label

In [16]:
results

[('In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.',
  '{"label": 1, "reasoning": "The sentence contains a future date \'08/2024\' and the phrase \'envision that\', which suggests a prediction about the future."}',
  1,
  "The sentence contains a future date '08/2024' and the phrase 'envision that', which suggests a prediction about the future.",
  'llama-3.1-8b-instant'),
 ('In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.',
  '{"label": 1, "reasoning": "Contains the future tense words \'envisions\' and a specific date \'08/2024\', indicating a statement about a potential future outcome regarding the touchdown rate at the New England Patriots"}',
  1,
  "Contains the future tense words 'envisions' and a specific date '08/2024', indicating a statement about a potential future outcome regarding the touchdown rate at the 

In [17]:
# groupby text

In [18]:
results_with_llm_label_df = pd.DataFrame(results, columns=['text', 'raw_response', 'llm_label', 'llm_reasoning', 'llm_name'])
results_with_llm_label_df

,text,raw_response,llm_label,llm_reasoning,llm_name
0,"In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.","{""label"": 1, ""reasoning"": ""The sentence contains a future date '08/2024' and the phrase 'envision that', which suggests a prediction about the future.""}",1,"The sentence contains a future date '08/2024' and the phrase 'envision that', which suggests a prediction about the future.",llama-3.1-8b-instant
1,"In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.","{""label"": 1, ""reasoning"": ""Contains the future tense words 'envisions' and a specific date '08/2024', indicating a statement about a potential future outcome regarding the touchdown rate at the New England Patriots""}",1,"Contains the future tense words 'envisions' and a specific date '08/2024', indicating a statement about a potential future outcome regarding the touchdown rate at the New England Patriots",llama-3.3-70b-versatile
2,"In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.","{""label"": 1, ""reasoning"": ""Contains the future tense phrase 'envisions that' and a specific date '08/2024', indicating a statement about a potential future outcome regarding the touchdown rate at the New England Patriots""}",1,"Contains the future tense phrase 'envisions that' and a specific date '08/2024', indicating a statement about a potential future outcome regarding the touchdown rate at the New England Patriots",llama-3.3-70b-instruct
3,Coach Rachel Thompson forecasts that the points per game at the New York Knicks potentially decrease in August 2024.,"{""label"": 1, ""reasoning"": ""Contains the future tense word 'will' and also a specific date 'August 2024' which indicates a prediction about the future.""}",1,Contains the future tense word 'will' and also a specific date 'August 2024' which indicates a prediction about the future.,llama-3.1-8b-instant
4,Coach Rachel Thompson forecasts that the points per game at the New York Knicks potentially decrease in August 2024.,"{""label"": 1, ""reasoning"": ""Contains the future tense words 'forecasts', 'potentially decrease', and a specific future date 'August 2024', indicating a prediction about the future.""}",1,"Contains the future tense words 'forecasts', 'potentially decrease', and a specific future date 'August 2024', indicating a prediction about the future.",llama-3.3-70b-versatile
5,Coach Rachel Thompson forecasts that the points per game at the New York Knicks potentially decrease in August 2024.,"{""label"": 1, ""reasoning"": ""Contains the future tense words 'forecasts', 'potentially decrease', and a specific future date 'August 2024'""}",1,"Contains the future tense words 'forecasts', 'potentially decrease', and a specific future date 'August 2024'",llama-3.3-70b-instruct
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,"{""label"": 1, ""reasoning"": ""Contains the future tense words 'will', 'potentially' and 'Q3 of 2027' which indicates a prediction about the future""}",1,"Contains the future tense words 'will', 'potentially' and 'Q3 of 2027' which indicates a prediction about the future",llama-3.1-8b-instant
7,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,"{""label"": 1, ""reasoning"": ""Contains the future tense words 'forecasts' and 'potentially decrease', along with a specific future date 'Q3 of 2027', indicating a prediction about the future revenue at Microsoft""}",1,"Contains the future tense words 'forecasts' and 'potentially decrease', along with a specific future date 'Q3 of 2027', indicating a prediction about the future revenue at Microsoft",llama-3.3-70b-versatile
8,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,"{""label"": 1, ""reasoning"

In [19]:
y_test_df.rename(columns={'Sentence Label' : 'Actual Label'}, inplace=True)

In [20]:
def get_llm_labels(df, model_name):
    filt_llama = (df['llm_name'] == model_name)
    filt_df = df[filt_llama]
    return filt_df['llm_label']


test_and_models_df = pd.concat([X_test_df.loc[:, :], y_test_df], axis=1)
# test_and_models_df.columns = ['Sentence', 'Actual Label']
for model in models:
    print(model.__name__())
    model_labels = get_llm_labels(results_with_llm_label_df, model.__name__())
    test_and_models_df[model.__name__()] = model_labels.to_numpy().ravel()
test_and_models_df

llama-3.1-8b-instant
llama-3.3-70b-versatile
llama-3.3-70b-instruct


,Base Sentence,Sentence Label,Embedding,Normalized Embeddings,Actual Label,llama-3.1-8b-instant,llama-3.3-70b-versatile,llama-3.3-70b-instruct
0,"In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.",1,[-2.95873899e-02 1.89737260e-01 -1.18197471e-01 -4.13006060e-02\n 1.10420622e-01 -4.95679379e-02 1.91853363e-02 3.95639287e-03\n 1.09274819e-01 1.97997952e+00 -1.85330868e-01 -4.54250947e-02\n 1.08167037e-01 2.54211240e-02 -9.49313119e-03 -6.84713572e-02\n 1.55237708e-02 1.00765920e+00 -6.00808859e-03 -4.15705629e-02\n -7.41642118e-02 -1.50733357e-02 1.34443464e-02 -8.06241706e-02\n -7.69194737e-02 1.14793696e-01 -2.15540364e-01 6.73782602e-02\n 3.61875258e-02 1.65809989e-02 -4.40017954e-02 2.61284299e-02\n 4.48011309e-02 -9.68891475e-03 3.49024460e-02 9.65972338e-03\n -2.66898591e-02 7.46985758e-03 -6.69648051e-02 -3.72406021e-02\n 7.12334365e-02 6.75264001e-02 7.85919055e-02 -7.79033080e-03\n 4.30065207e-02 -5.40831983e-02 -1.17614612e-01 -2.35915799e-02\n -4....,[ 0.94400704 -0.8645446 -2.054663 -0.3882444 0.8747154 -0.2001761\n 0.3727082 0.06485146 0.7578419 0.74855906 1.5445073 -0.9450778\n 0.8223399 0.4856363 0.34803256 -0.53150874 0.5283372 -0.55164915\n 1.8045051 -0.24819341 -2.180726 -1.0896391 0.08680368 0.22307044\n -1.5891142 0.5755748 -1.1823585 0.6673478 0.25934806 -0.86385673\n -0.6393052 0.24252476 0.7617 -1.2495198 -0.25059918 0.46805078\n -0.01315914 -0.6343914 -0.95103335 -0.23913547 1.234197 0.26890773\n -0.0123065 0.36895898 0.34772688 -0.8687144 -0.7102191 0.17720816\n -1.3530828 0.6399917 0.0253048 1.1315668 -0.82290274 0.01780647\n 0.33573022 0.84572476 0.5561635 0.630834 0.5291998 1.5564367\n 0.74758047 1.1630359 0.18509752 -1.5357434 -0.7491160...,1,1,1,1
1,Coach Rachel Thompson forecasts that the points per game at the New York Knicks potentially decrease in August 2024.,1,[-7.31271803e-02 2.57879347e-01 -1.00163361e-02 -2.91225519e-02\n 6.20316342e-02 -1.89948324e-02 2.83652991e-02 -2.36207973e-02\n 1.25035286e-01 1.68172586e+00 -3.08204234e-01 3.09939720e-02\n 5.19324541e-02 -4.09708358e-03 5.32346480e-02 -9.79294349e-03\n -1.77208520e-02 9.74500477e-01 -6.05918542e-02 -1.19134128e-01\n 6.85500652e-02 -1.13969576e-02 1.40849143e-01 -1.98314548e-01\n -2.55904496e-02 2.24934310e-01 -2.85059363e-01 1.34536862e-01\n -5.01615815e-02 1.06635571e-01 -1.34736553e-01 6.85500056e-02\n 9.54245403e-03 1.24958850e-01 8.98100063e-03 5.74925914e-02\n 3.36430110e-02 3.12536396e-02 -2.97347642e-02 -5.55007998e-03\n -7.19834417e-02 1.25202790e-01 3.19119208e-02 -8.32804516e-02\n -8.05644970e-03 -9.72918496e-02 -1.58620656e-01 7.15503991e-02\n -3....,[ 0.4060981 0.06459576 -0.33290786 -0.18525055 0.23426409 0.28097287\n 0.5438155 -0.29924256 0.9618827 -0.30452695 -0.16134328 0.04894692\n -0.02327521 -0.02009717 1.0117191 0.54240227 -0.07506002 -0.78002316\n 0.8962295 -1.5484266 0.80796844 -1.033549 2.02265 -1.4457239\n -0.84145296 2.089933 -2.1967309 1.8129327 -1.2062268 0.43286043\n -2.3416817 0.92772835 0.19314055 0.6750809 -0.73656416 1.299064\n 0.8058958 -0.26324862 -0.31853175 0.24293266 -1.204331 1.1874502\n -0.641938 -0.9460187 -0.5985572 -1.5994209 -1.2832663 1.3151182\n -0.65960604 -0.5158133 0.5862492 -0.48699406 0.34597123 -1.3362038\n -1.5870239 -0.51357347 0.21139102 1.4604084 1.0736531 -0.19801451\n -1.9606705 1.1726837 -0.5702353 -2.2427983 0.5018804 ...,1,1,1,1
2,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,1,[-2.24677563e-01 2.28302136e-01 -3.20941433e-02 -1.78482141e-02\n -5.66348508e-02 2.05190871e-02 -3.81574295e-02 1.41838089e-01\n 1.64982721e-01 1.45077002e+00 -3.47716570e-01 4.82952334e-02\n 1.09031864e-01 4.18408737e-02 6.93667382e-02 7.94899929e-03\n -2.80997157e-02 1.02238500e+00 -1.17930911e-01 -1.29385918e-01\n 4.74877171e-02 1.93608850e-01 6.20140657e-02 -1.79264918e-01\n 6.11357428e-02 1.66994065e-01 -9.25592855e-02 5.74763976e-02\n 2.44720709e-02 6.20023720e-02 -3.919312

## Save Output

In [21]:
# save_path = os.path.join(combine_data_path )
DataProcessing.save_to_file(test_and_models_df, combine_data_path, 'llm_classifiers', '.csv')

Saved to: 
	/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_classification_experiments-v2/../data/combined_generated_fin_phrase_bank/llm_classifiers-v1.csv


## Evaluation

In [22]:
print("======= EVALUATION/RESULTS =======")

======= EVALUATION/RESULTS =======


In [23]:
get_metrics = EvaluationMetric()
get_metrics

In [24]:
actual_label = test_and_models_df['Actual Label'].values
for ml_model in models:
    ml_model_name = ml_model.__name__()
    print(f"Actual Label:\t\t{actual_label}")
    ml_model_predictions = test_and_models_df[ml_model_name].values
    print(f"{ml_model_name}:\t\t{ml_model_predictions}")
    get_metrics.eval_classification_report(y_test_df, ml_model_predictions)

Actual Label:		[1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1]
llama-3.1-8b-instant:		[1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0]
              precision    recall  f1-score   support

           0       0.62      1.00      0.77         5
           1       1.00      0.84      0.91        19

    accuracy                           0.88        24
   macro avg       0.81      0.92      0.84        24
weighted avg       0.92      0.88      0.88        24

Actual Label:		[1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1]
llama-3.3-70b-versatile:		[1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       1.00      0.89      0.94        19

    accuracy                           0.92        24
   macro avg       0.86      0.95      0.89        24
weighted avg       0.94      0.92      0.92        24

Actual Label:		[1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1]
ll